# Latent Dirichlet Allocation
---
  
**LDA**  
- Generative Probabilistic Topic Model. 
- Assumes documents are a mixture of topics and that each word in the document is attributable to a topic  
- The topic probabilities provide an explicit representation of a document

## Env Preparation

In [1]:
RUNTIMES = ['cluster', 'local'] # Modify the paths from the coonfig.yaml
RUNTIME = RUNTIMES[0]

In [2]:
import os
import sys
# os.chdir('/Users/pabloruizruiz/OneDrive/Courses/NLP_Stanford/Complementary_Courses/Document_Clustering')
os.chdir('/app/')
print('Workdir: ', os.getcwd())
from os.path import join as JP
sys.path.append(os.getcwd())
sys.path.append(JP(os.getcwd(),'utils'))
sys.path.append(JP(os.getcwd(),'scripts'))

Workdir:  /app


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()
%matplotlib notebook
np.set_printoptions(precision=3)
pd.options.display.float_format = '{:,.3f}'.format

In [5]:
import pickle
from pprint import pprint
from collections import defaultdict

from utils.nlp_utils import preproces
from utils.general import parse_yaml, ensure_directories

from scripts.catalog import (
    Catalog, Document, Corpus,
    load_catalog, load_corpus)

config = parse_yaml('config.yaml')
paths = config['paths']
ensure_directories(paths)
paths

{'results': 'results/',
 'catalog': 'data/catalog',
 'checkpoints': 'checkpoints/',
 'stopwords': 'data/stopwords/',
 'data': 'data/',
 'images': 'images/'}

In [8]:
from nltk.corpus import stopwords
SW = stopwords.words('english')
# with open(JP(paths['stopwords'],'nltk_stopwords.pkl'), 'rb') as f:
#     SW = pickle.load(f)

## Data

In [9]:
name = 'bbc-text-processed.csv'

In [10]:
data = pd.read_csv(JP('data',name), index_col=0)
data.head()

,category,text,lenght,processed
0,tech,tv future in the hands of viewers with home th...,806,tv future hand viewer system plasma high defin...
1,business,worldcom boss left books alone former worldc...,332,left book ebber accuse oversee 11bn fraud acco...
2,sport,tigers wary of farrell gamble leicester say ...,270,wary farrell gamble rush make bid decide switc...
3,sport,yeading face newcastle in fa cup premiership s...,390,yeade face premiership face trip leader yeade ...
4,entertainment,ocean s twelve raids box office ocean s twelve...,287,raid ocean crime go number office chart take w...


## TFIDF

In [11]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
EMBED_SIZE = 10000 
NUM_CLUSTERS = data['category'].nunique()
WORDS_PER_CLUSTER = None
print(NUM_CLUSTERS)

5


In [13]:
documents = [Document() for i in range(data.shape[0])]
for d in range(len(documents)):
    documents[d].processed_text = data['processed'][d]

In [14]:
catalog = Catalog()
catalog.documents = documents

In [15]:
vectorizer = TfidfVectorizer(
    min_df=.05,
    max_df=.8,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    max_features=EMBED_SIZE,
    ngram_range=(1,3),
    lowercase=True,
    stop_words=SW)

In [16]:
_ = catalog.collect_corpus(attr='processed_text', form=list)
tfidf = catalog.to_matrix(
    vectorizer=vectorizer,
    modelname='TFIDF',
    max_docs=None)
print(tfidf.representation.shape)
tfidf.representation.head()

(2225, 460)


,able,accept,access,accord,account,accuse,act,action,add,admit,...,week,weekend,win,winner,woman,work,world,write,year,young
0,0.000,0.000,0.000,0.037,0.000,0.000,0.000,0.000,0.027,0.000,...,0.000,0.000,0.000,0.000,0.000,0.029,0.000,0.000,0.038,0.000
1,0.000,0.000,0.000,0.000,0.000,0.150,0.000,0.000,0.000,0.299,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.106,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.239,0.066,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.115,0.000,0.000,0.000,0.000,0.000,0.000,...,0.094,0.316,0.087,0.000,0.000,0.000,0.000,0.000,0.000,0.000


## Sklearn Topic Modelling

- 1: Latent Dirichlet Allocation
- 2: Non Negative Matrix Factorization

In [17]:
NUM_TOPICS = NUM_CLUSTERS

In [18]:
def selected_topics(model, vectorizer, top_n=10):
    ''' Functions for printing keywords for each topic '''
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]]) 

### Latente Dirichlet Allocation

In [17]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
lda = LatentDirichletAllocation(
    n_components=NUM_TOPICS, 
    learning_method='batch', 
    max_iter=100, 
    batch_size=128, 
    random_state=2019, 
    verbose=0)
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=100,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=2019,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

**Load Pretrained Model**

In [19]:
with open(JP(paths['checkpoints'], 'lda_sklearn.pkl'), 'rb') as obj:
    lda = pickle.load(obj)
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=True)

**Train Model**

In [32]:
print('Latent Semantion Allocation')
data_lda = lda.fit_transform(tfidf.representation)

Latent Semantion Allocation
iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100
iteration: 16 of max_iter: 100
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100
iteration: 25 of max_iter: 100
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100
iteration: 29 of max_iter: 100
iteration: 30 of max_iter: 100
iteration: 31 of max_iter: 100
iteration: 32 of max

In [33]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer, 8)

LDA Model:
Topic 0:
['game', 'win', 'play', 'player', 'match', 'team', 'club', 'injury']
Topic 1:
['government', 'election', 'party', 'labour', 'people', 'plan', 'tory', 'law']
Topic 2:
['technology', 'people', 'mobile', 'phone', 'user', 'computer', 'service', 'software']
Topic 3:
['film', 'award', 'star', 'music', 'include', 'year', 'win', 'release']
Topic 4:
['company', 'sale', 'share', 'firm', 'market', 'price', 'year', 'rise']


**Save Model**

In [34]:
with open(JP(paths['checkpoints'], 'lda_sklearn.pkl'), 'wb') as obj:
    pickle.dump(lda,obj)

#### Visualization

In [20]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [27]:
pyLDAvis.sklearn.prepare(lda, tfidf.dtm_sparse, tfidf.mapping, mds='tsne')

PreparedData(topic_coordinates=            x        y  topics  cluster   Freq
topic                                         
2     140.918 -131.135       1        1 33.218
4     110.959  -29.697       2        1 32.745
3      32.985 -149.124       3        1 27.919
0      25.530   32.666       4        1  3.059
1     -27.752  -62.907       5        1  3.059, topic_info=    Category   Freq        Term  Total  loglift  logprob
151  Default 78.000        film 78.000   30.000   30.000
127  Default 49.000    election 49.000   29.000   29.000
166  Default 82.000        game 82.000   28.000   28.000
452  Default 85.000         win 85.000   27.000   27.000
176  Default 62.000  government 62.000   26.000   26.000
..       ...    ...         ...    ...      ...      ...
132   Topic5  0.817       event 27.030   -0.013   -6.130
352   Topic5  0.817    research 18.076    0.390   -6.130
287   Topic5  0.816        open 37.408   -0.338   -6.130
295   Topic5  0.816      people 78.890   -1.084   -6.130
151   Topic5  0.816        film 78.048   -1.073   -6.130

[286 rows x 6 columns], token_table=      Topic  Freq    Term
term                     
1         1 0.138  accept
1         2 0.208  accept
1         3 0.554  accept
1         4 0.069  accept
1         5 0.069  accept
...     ...   ...     ...
458       1 0.403    year
458       2 0.384    year
458       3 0.197    year
458       4 0.009    year
458       5 0.009    year

[859 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 1, 2])

## Non-Negative Matrix Factorization

In [37]:
nmf = NMF(n_components=NUM_TOPICS)
nmf

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [28]:
print('Non-Negative Matrix Factorization')
data_nmf = nmf.fit_transform(tfidf.representation)

Non-Negative Matrix Factorization


In [29]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer, 8)

NMF Model:
Topic 0:
['sale', 'year', 'rise', 'company', 'market', 'growth', 'share', 'price']
Topic 1:
['game', 'win', 'play', 'player', 'match', 'team', 'final', 'injury']
Topic 2:
['film', 'award', 'star', 'win', 'movie', 'include', 'director', 'year']
Topic 3:
['election', 'party', 'labour', 'government', 'tory', 'plan', 'tax', 'leader']
Topic 4:
['people', 'mobile', 'phone', 'technology', 'user', 'music', 'service', 'computer']


**Save Model**

In [30]:
with open(JP(paths['checkpoints'], 'NNMF.pkl'), 'wb') as obj:
    pickle.dump(nmf,obj)

**Load Model**

In [38]:
with open(JP(paths['checkpoints'], 'NNMF.pkl'), 'rb') as obj:
    nmf = pickle.load(obj)
nmf

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)